# Árvores II - Tarefa 2

### 1. Carregar as bases

Vamos carregar as bases lidas na tarefa passada. Se você salvou essas bases em arquivo texto, basta fazer a leitura com o comando ```pd.read_csv``` das seguintes bases:

- X_train
- Y_train
- X_test
- Y_test

Não se esqueça de considerar a leitura dos índices dos arquivos no ```read_csv()```!

In [25]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import os

In [13]:
# Carregar os arquivos salvos anteriormente
train_data = pd.read_csv("train_data.csv", index_col=[0, 1])  # Índice duplo
test_data = pd.read_csv("test_data.csv", index_col=[0, 1])  # Índice duplo

# Caminho base para os arquivos originais
base_path = r'C:\Users\Zghos\Desktop\CursoEBAC\CientistaDados\Mod17\UCI HAR Dataset'

# Carregar os rótulos das atividades (y_train e y_test)
y_train = pd.read_csv(os.path.join(base_path, 'train/y_train.txt'), header=None, names=['activity'])
y_test = pd.read_csv(os.path.join(base_path, 'test/y_test.txt'), header=None, names=['activity'])

# Combinar os rótulos às bases originais
train_data['activity'] = y_train.values
test_data['activity'] = y_test.values

# Separar variáveis explicativas (X) e rótulos (y)
X_train = train_data.drop(columns=['activity'])
y_train = train_data['activity']

X_test = test_data.drop(columns=['activity'])
y_test = test_data['activity']

# Verificar dimensões
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

X_train shape: (7352, 3)
y_train shape: (7352,)
X_test shape: (2947, 3)
y_test shape: (2947,)


### 2. Divisão da base em Treino, Validação e Teste

A base já se encontra dividida em Treino e Validação. O que vamos fazer então é extrair uma base de Validação da base de Treino.

Extraia 25% da base de treino como base de validação.

In [15]:
# Divisão da base de treino em treino e validação (75% treino, 25% validação)
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.25, random_state=42, stratify=y_train
)

# Verificar as dimensões das novas bases
print("Dimensões após divisão:")
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_val shape:", X_val.shape)
print("y_val shape:", y_val.shape)

Dimensões após divisão:
X_train shape: (5514, 3)
y_train shape: (5514,)
X_val shape: (1838, 3)
y_val shape: (1838,)


### 3. Melhores 3 variáveis

Rode uma árvore com profundidade máxima igual a 4 para prever a atividade humana com todas as variáveis.
Observe a importância das variáveis e considere as 3 variáveis com maior importância para os próximos passos.
Dica: utilize o atributo ```clf.feature_importances_``` da árvore treinada.

In [17]:
# Treinar a árvore de decisão com profundidade máxima igual a 4
clf = DecisionTreeClassifier(max_depth=4, random_state=42)
clf.fit(X_train, y_train)

# Obter as importâncias das variáveis
importances = clf.feature_importances_

# Criar um dataframe para organizar as variáveis e suas importâncias
importance_df = pd.DataFrame({
    'Variable': X_train.columns,
    'Importance': importances
})

# Ordenar pelas variáveis mais importantes
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# Selecionar as 3 variáveis mais importantes
top_3_variables = importance_df.head(3)

# Exibir as 3 variáveis mais importantes
print("Top 3 variáveis mais importantes:")
print(top_3_variables)

Top 3 variáveis mais importantes:
            Variable  Importance
0  tBodyAcc-mean()-X    0.679333
1  tBodyAcc-mean()-Y    0.184846
2  tBodyAcc-mean()-Z    0.135822


### 4. Construa uma árvore com as 3 melhores variáveis

Utilizando as três variáveis encontradas acima, construa uma árvore de decisão. Encontre o melhor ```ccp_alpha``` utilizando a base de validação, conforme a estrutura que vimos em aula.

In [21]:
# Selecionar as 3 variáveis mais importantes
selected_variables = ['tBodyAcc-mean()-X', 'tBodyAcc-mean()-Y', 'tBodyAcc-mean()-Z']

X_train_filtered = X_train[selected_variables]
X_val_filtered = X_val[selected_variables]
X_test_filtered = X_test[selected_variables]

In [23]:
# Ajustar uma árvore inicial (sem poda) para encontrar os valores de ccp_alpha
clf = DecisionTreeClassifier(random_state=42)
path = clf.cost_complexity_pruning_path(X_train_filtered, y_train)

# Obter os valores de ccp_alpha
ccp_alphas = path.ccp_alphas

# Filtrar valores únicos e positivos
ccp_alphas = sorted(set(alpha for alpha in ccp_alphas if alpha > 0))
print("Valores de ccp_alpha:", ccp_alphas)

Valores de ccp_alpha: [np.float64(0.00012090436464756376), np.float64(0.00012090436464756379), np.float64(0.00012954039069381834), np.float64(0.00013601741022850925), np.float64(0.0001450852375770765), np.float64(0.00014735219441421835), np.float64(0.0001511304558094547), np.float64(0.00015113045580945474), np.float64(0.00015168002110330735), np.float64(0.00015204639796587567), np.float64(0.0001541530649256438), np.float64(0.00015544846883258203), np.float64(0.00015544846883258206), np.float64(0.00015662610874798033), np.float64(0.00015868697859992744), np.float64(0.00015868697859992747), np.float64(0.000160430791551575), np.float64(0.00016120581953008503), np.float64(0.0001612854666741216), np.float64(0.0001619254883672729), np.float64(0.00016322089227421106), np.float64(0.0001632208922742111), np.float64(0.00016322089227421114), np.float64(0.0001640844948788366), np.float64(0.0001645350701508151), np.float64(0.0001648695881557688), np.float64(0.00016494809748346197), np.float64(0.000

In [27]:
# Listas para armazenar árvores e acurácias
trees = []
val_accuracies = []

# Ajustar árvores para diferentes valores de ccp_alpha
for alpha in ccp_alphas:
    tree = DecisionTreeClassifier(ccp_alpha=alpha, random_state=42)
    tree.fit(X_train_filtered, y_train)
    trees.append(tree)
    
    # Calcular a acurácia na base de validação
    val_acc = accuracy_score(y_val, tree.predict(X_val_filtered))
    val_accuracies.append(val_acc)

# Exibir as acurácias
for alpha, acc in zip(ccp_alphas, val_accuracies):
    print(f"ccp_alpha: {alpha:.5f}, Acurácia na Validação: {acc:.2f}")

ccp_alpha: 0.00012, Acurácia na Validação: 0.39
ccp_alpha: 0.00012, Acurácia na Validação: 0.40
ccp_alpha: 0.00013, Acurácia na Validação: 0.39
ccp_alpha: 0.00014, Acurácia na Validação: 0.40
ccp_alpha: 0.00015, Acurácia na Validação: 0.40
ccp_alpha: 0.00015, Acurácia na Validação: 0.40
ccp_alpha: 0.00015, Acurácia na Validação: 0.40
ccp_alpha: 0.00015, Acurácia na Validação: 0.40
ccp_alpha: 0.00015, Acurácia na Validação: 0.40
ccp_alpha: 0.00015, Acurácia na Validação: 0.40
ccp_alpha: 0.00015, Acurácia na Validação: 0.40
ccp_alpha: 0.00016, Acurácia na Validação: 0.40
ccp_alpha: 0.00016, Acurácia na Validação: 0.40
ccp_alpha: 0.00016, Acurácia na Validação: 0.40
ccp_alpha: 0.00016, Acurácia na Validação: 0.40
ccp_alpha: 0.00016, Acurácia na Validação: 0.40
ccp_alpha: 0.00016, Acurácia na Validação: 0.40
ccp_alpha: 0.00016, Acurácia na Validação: 0.40
ccp_alpha: 0.00016, Acurácia na Validação: 0.40
ccp_alpha: 0.00016, Acurácia na Validação: 0.40
ccp_alpha: 0.00016, Acurácia na Validaçã

In [31]:
# Encontrar o índice do melhor alpha
best_index = val_accuracies.index(max(val_accuracies))
best_ccp_alpha = ccp_alphas[best_index]
best_tree = trees[best_index]

print(f"Melhor ccp_alpha: {best_ccp_alpha:.5f}")
print(f"Melhor acurácia na base de validação: {val_accuracies[best_index]:.2f}")

Melhor ccp_alpha: 0.00078
Melhor acurácia na base de validação: 0.46


### 5. Avaliação do modelo

Avalie a árvore encontrada no item anterior na base de testes.

In [33]:
# Avaliar a melhor árvore na base de teste
test_accuracy = accuracy_score(y_test, best_tree.predict(X_test_filtered))
print(f"Acurácia na base de teste com o melhor ccp_alpha: {test_accuracy:.2f}")

Acurácia na base de teste com o melhor ccp_alpha: 0.44
